In [1]:
%display typeset

In [2]:
import math

def calcula_prob(v, u):
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    p = [0, 0]
    p[0] = math.exp(u[0]*v[1])/(math.exp(u[0]*v[1]) + math.exp(u[1]*v[0]))
    p[1] = 1 - p[0]
    
    return p

def gera_parametro(p):
    # p = [p0, p1], onde p0 = x/(x + y) e p1 = y/(x + y), com x = e^(u0*v1) e y = e^(v0*u1)
    # queremos encontrar v = [v0, v1] e u = [u0, u1]
    x = math.log(p[0]) # faz x = ln(x) - ln(x + y) = ln(x) = u0*v1
    y = math.log(p[1]) # faz y = ln(y) - ln(x + y) = ln(y) = v0*u1
    
    # considerando u0 = 1 = v0, temos que 
    u = [1, x]
    v = [1, y]
    
    '''
    tentativa de normalizar
    
    v = [1/math.sqrt(1 + x**2), x/math.sqrt(1 + x**2)]
    u = [1/math.sqrt(1 + y**2), y/math.sqrt(1 + y**2)]
    '''
    
    return [u, v]

In [3]:
v1 = [   1,               0]
v2 = [-0.5,  math.sqrt(3)/2]
v3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre v1 e v2:", calcula_prob(v1, v2))
print("Jogo entre v2 e v3:", calcula_prob(v2, v3))
print("Jogo entre v3 e v1:", calcula_prob(v3, v1))

Jogo entre v1 e v2: [0.2960820052793571, 0.7039179947206429]
Jogo entre v2 e v3: [0.2960820052793571, 0.7039179947206429]
Jogo entre v3 e v1: [0.2960820052793571, 0.7039179947206429]


In [4]:
p = calcula_prob(v1, v2)
param = gera_parametro(p)
q = calcula_prob(param[0], param[1])
print("Probabilidades com p:", p)
print("Parâmetros:", param)
print("Probabilidades os parâmetros encontrados:", q)

Probabilidades com p: [0.2960820052793571, 0.7039179947206429]
Parâmetros: [[1, -1.2171188181652253], [1, -0.3510934143807867]]
Probabilidades os parâmetros encontrados: [0.2960820052793571, 0.7039179947206429]


In [5]:
import math

def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    logit = v[1]/u[0] - u[1]/v[0]
    p = [0, 0]
    p[0] = 1/(1 + math.exp(-logit))
    p[1] = 1 - p[0]
    
    return p

def verossimilhanca(resultados, jogadores):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    # jogadores é a lista de parâmetros dos jogadores, assim parâmetros[0]
    # contém os parâmetros do jogador de ID = 0
    
    log_ver = 0
    p = [0, 0]
    
    for i in range(len(resultados)):
        p = calcula_prob(jogadores[resultados[i][0]], jogadores[resultados[i][1]])
        log_ver += math.log(p[0])
    
    return - log_ver

In [6]:
P1 = [   1,               0]
P2 = [-0.5,  math.sqrt(3)/2]
P3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre P1 e P2:", calcula_prob(P1, P2))
print("Jogo entre P2 e P3:", calcula_prob(P2, P3))
print("Jogo entre P3 e P1:", calcula_prob(P3, P1))

Jogo entre P1 e P2: [0.2960820052793571, 0.7039179947206429]
Jogo entre P2 e P3: [0.030351090329424395, 0.9696489096705756]
Jogo entre P3 e P1: [0.2960820052793571, 0.7039179947206429]


In [7]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [P1, P2, P3]

verossimilhanca(resultados, jogadores)

5.959981695062614

In [8]:
import numpy as np

def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna p, a prob do jogador 1 ganhar do jogador 2
    logit = v[1]/u[0] - u[1]/v[0]
    
    # coloquei esse try pois tem vezes que essa exponencial resulta em um valor muito grande
    # gerando o OverflowError
    try:
        p = 1/(1 + math.exp(-logit))
    except OverflowError:
        p = float(1e-31)
    
    return p

def verossimilhanca(jogadores, resultados):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # jogadores é a lista de parâmetros dos jogadores. Nela os parâmetros
    # do jogador de ID = i estão em parâmetros[2*i] e em parâmetros[2*i + 1]
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    
    log_ver = 0
    
    for i in range(len(resultados)):
        indice_1 = 2*resultados[i][0]
        indice_2 = 2*resultados[i][1]
        param_1 = [jogadores[indice_1], jogadores[indice_1 + 1]]
        param_2 = [jogadores[indice_2], jogadores[indice_2 + 1]]
        p = calcula_prob(param_1, param_2)
        try:
            add = math.log(p)
        except ValueError:
            return np.inf
        log_ver += math.log(p)
    
    return - log_ver

In [9]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [1, 0, -0.5, math.sqrt(3)/2, -0.5, -math.sqrt(3)/2]

verossimilhanca(jogadores, resultados)

5.959981695062614

In [10]:
cons = ({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[0]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[4]})

In [11]:
from scipy.optimize import minimize

results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

In [12]:
results.x = results.x * (1/min(abs(results.x)))
print("Os parâmetros encontrados são:\n", results.x)

Os parâmetros encontrados são:
 [503.65347438 250.76030715  40.62255516 -42.51804589 548.74536369
   1.        ]


In [13]:
results.fun

2.3371095574672474

In [14]:
cons = []
jogadores = []
for i in range(3):
    cons.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    jogadores.append(1)
    jogadores.append(0)

In [15]:
cons = tuple(cons)

In [16]:
results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

In [17]:
results.x

array([ 2.99009532e-06,  1.69032020e-07,  1.67042410e+00, -3.08439143e-01,
        1.67042279e+00,  3.08440896e-01])

In [18]:
results.fun

1.4201971731668042

In [19]:
from openpyxl import load_workbook
import numpy as np

def pega_jogos(file):
    games = load_workbook(file)

    jogos = games['Planilha1']

    resultados = []

    for row in jogos.iter_rows():
        jogo = []
        for cell in row:
            if cell.value != 'w' and cell.value != 'l':
                jogo.append(cell.value)
        resultados.append(jogo)
    
    if resultados[0] == []:
        resultados.pop(0)
    return resultados

def conta_jogadores(resultados):
    contagem = 0
    for i in range(len(resultados)):
        for j in resultados[i]:
            if j > contagem:
                contagem = j
    
    return contagem + 1

def cria_restricoes(contagem):
    restricoes = []
    for i in range(contagem):
        restricoes.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    
    restricoes = tuple(restricoes)
    return restricoes

def cria_jogadores(contagem):
    jogadores = []
    for i in range(contagem):
        jogadores.append(np.random.random())
        jogadores.append(np.random.random())
    
    return jogadores

def calcula_parametros(file):
    resultados = pega_jogos(file)
    contagem = conta_jogadores(resultados)
    cons = cria_restricoes(contagem)
    jogadores = cria_jogadores(contagem)
    results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)
    
    return results

In [20]:
# results = calcula_parametros('matches.xlsx')

In [21]:
# results.fun

In [22]:
resultados = pega_jogos("matches.xlsx")

In [23]:
# resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
# jogadores = [1, 0, -0.5, math.sqrt(3)/2, -0.5, -math.sqrt(3)/2]

In [24]:
contagem = conta_jogadores(resultados)
contagem

722

In [25]:
restricoes = cria_restricoes(contagem)
jogadores = cria_jogadores(contagem)

In [26]:
# restricoes[0]

In [27]:
if contagem <= 10:
    jogadores

In [28]:
if contagem <= 10:
    restricoes

In [29]:
# foi a melhor forma de fazer as restrições que achei

cons = ({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[0]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[4]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[6]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[8]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[10]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[12]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[14]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[16]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[18]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[20]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[22]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[24]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[26]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[28]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[30]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[32]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[34]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[36]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[38]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[40]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[42]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[44]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[46]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[48]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[50]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[52]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[54]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[56]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[58]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[60]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[62]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[64]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[66]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[68]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[70]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[72]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[74]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[76]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[78]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[80]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[82]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[84]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[86]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[88]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[90]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[92]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[94]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[96]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[98]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[100]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[102]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[104]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[106]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[108]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[110]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[112]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[114]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[116]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[118]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[120]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[122]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[124]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[126]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[128]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[130]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[132]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[134]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[136]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[138]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[140]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[142]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[144]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[146]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[148]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[150]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[152]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[154]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[156]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[158]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[160]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[162]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[164]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[166]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[168]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[170]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[172]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[174]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[176]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[178]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[180]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[182]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[184]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[186]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[188]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[190]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[192]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[194]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[196]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[198]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[200]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[202]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[204]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[206]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[208]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[210]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[212]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[214]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[216]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[218]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[220]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[222]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[224]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[226]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[228]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[230]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[232]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[234]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[236]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[238]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[240]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[242]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[244]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[246]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[248]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[250]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[252]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[254]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[256]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[258]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[260]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[262]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[264]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[266]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[268]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[270]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[272]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[274]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[276]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[278]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[280]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[282]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[284]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[286]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[288]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[290]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[292]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[294]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[296]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[298]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[300]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[302]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[304]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[306]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[308]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[310]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[312]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[314]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[316]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[318]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[320]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[322]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[324]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[326]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[328]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[330]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[332]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[334]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[336]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[338]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[340]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[342]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[344]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[346]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[348]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[350]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[352]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[354]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[356]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[358]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[360]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[362]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[364]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[366]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[368]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[370]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[372]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[374]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[376]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[378]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[380]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[382]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[384]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[386]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[388]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[390]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[392]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[394]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[396]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[398]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[400]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[402]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[404]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[406]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[408]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[410]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[412]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[414]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[416]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[418]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[420]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[422]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[424]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[426]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[428]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[430]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[432]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[434]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[436]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[438]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[440]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[442]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[444]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[446]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[448]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[450]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[452]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[454]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[456]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[458]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[460]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[462]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[464]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[466]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[468]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[470]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[472]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[474]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[476]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[478]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[480]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[482]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[484]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[486]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[488]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[490]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[492]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[494]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[496]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[498]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[500]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[502]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[504]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[506]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[508]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[510]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[512]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[514]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[516]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[518]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[520]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[522]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[524]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[526]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[528]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[530]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[532]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[534]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[536]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[538]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[540]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[542]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[544]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[546]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[548]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[550]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[552]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[554]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[556]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[558]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[560]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[562]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[564]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[566]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[568]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[570]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[572]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[574]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[576]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[578]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[580]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[582]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[584]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[586]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[588]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[590]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[592]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[594]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[596]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[598]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[600]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[602]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[604]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[606]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[608]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[610]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[612]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[614]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[616]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[618]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[620]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[622]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[624]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[626]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[628]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[630]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[632]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[634]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[636]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[638]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[640]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[642]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[644]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[646]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[648]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[650]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[652]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[654]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[656]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[658]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[660]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[662]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[664]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[666]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[668]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[670]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[672]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[674]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[676]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[678]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[680]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[682]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[684]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[686]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[688]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[690]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[692]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[694]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[696]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[698]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[700]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[702]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[704]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[706]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[708]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[710]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[712]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[714]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[716]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[718]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[720]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[722]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[724]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[726]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[728]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[730]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[732]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[734]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[736]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[738]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[740]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[742]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[744]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[746]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[748]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[750]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[752]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[754]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[756]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[758]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[760]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[762]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[764]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[766]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[768]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[770]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[772]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[774]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[776]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[778]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[780]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[782]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[784]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[786]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[788]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[790]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[792]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[794]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[796]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[798]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[800]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[802]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[804]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[806]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[808]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[810]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[812]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[814]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[816]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[818]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[820]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[822]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[824]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[826]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[828]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[830]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[832]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[834]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[836]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[838]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[840]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[842]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[844]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[846]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[848]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[850]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[852]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[854]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[856]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[858]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[860]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[862]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[864]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[866]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[868]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[870]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[872]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[874]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[876]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[878]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[880]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[882]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[884]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[886]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[888]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[890]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[892]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[894]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[896]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[898]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[900]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[902]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[904]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[906]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[908]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[910]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[912]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[914]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[916]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[918]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[920]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[922]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[924]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[926]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[928]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[930]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[932]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[934]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[936]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[938]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[940]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[942]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[944]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[946]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[948]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[950]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[952]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[954]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[956]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[958]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[960]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[962]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[964]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[966]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[968]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[970]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[972]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[974]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[976]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[978]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[980]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[982]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[984]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[986]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[988]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[990]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[992]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[994]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[996]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[998]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1000]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1002]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1004]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1006]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1008]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1010]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1012]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1014]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1016]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1018]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1020]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1022]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1024]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1026]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1028]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1030]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1032]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1034]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1036]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1038]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1040]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1042]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1044]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1046]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1048]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1050]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1052]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1054]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1056]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1058]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1060]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1062]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1064]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1066]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1068]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1070]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1072]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1074]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1076]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1078]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1080]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1082]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1084]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1086]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1088]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1090]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1092]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1094]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1096]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1098]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1100]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1102]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1104]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1106]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1108]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1110]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1112]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1114]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1116]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1118]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1120]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1122]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1124]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1126]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1128]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1130]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1132]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1134]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1136]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1138]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1140]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1142]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1144]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1146]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1148]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1150]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1152]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1154]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1156]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1158]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1160]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1162]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1164]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1166]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1168]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1170]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1172]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1174]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1176]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1178]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1180]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1182]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1184]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1186]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1188]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1190]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1192]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1194]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1196]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1198]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1200]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1202]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1204]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1206]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1208]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1210]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1212]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1214]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1216]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1218]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1220]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1222]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1224]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1226]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1228]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1230]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1232]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1234]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1236]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1238]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1240]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1242]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1244]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1246]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1248]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1250]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1252]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1254]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1256]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1258]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1260]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1262]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1264]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1266]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1268]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1270]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1272]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1274]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1276]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1278]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1280]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1282]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1284]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1286]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1288]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1290]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1292]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1294]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1296]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1298]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1300]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1302]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1304]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1306]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1308]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1310]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1312]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1314]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1316]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1318]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1320]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1322]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1324]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1326]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1328]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1330]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1332]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1334]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1336]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1338]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1340]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1342]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1344]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1346]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1348]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1350]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1352]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1354]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1356]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1358]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1360]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1362]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1364]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1366]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1368]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1370]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1372]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1374]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1376]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1378]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1380]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1382]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1384]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1386]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1388]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1390]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1392]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1394]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1396]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1398]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1400]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1402]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1404]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1406]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1408]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1410]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1412]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1414]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1416]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1418]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1420]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1422]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1424]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1426]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1428]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1430]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1432]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1434]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1436]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1438]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1440]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[1442]})

In [ ]:
results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)
results.x

In [ ]:
results.fun

In [ ]:
len(resultados)

In [ ]:
P = list(var('P_%d' % i) for i in range(contagem)) # primeiro parâmetro de cada atleta
Q = list(var('Q_%d' % i) for i in range(contagem)) # segundo parâmetro de cada atleta

In [38]:
np.inf

inf